In [ ]:
from zipfile import ZipFile
import re
from datetime import date, timedelta

WORDLE = "Wordle"
BRONZE = 1
SILVER = BRONZE * 100
GOLD = SILVER * 100
MS = 86400000
MEDALS = [GOLD, SILVER, BRONZE]
MAX = 100

MEDAL_SYMBOLS = {
  GOLD: '🥇',
  SILVER: '🥈',
  BRONZE: '🥉'
};

MEDAL_NAMES = {
  GOLD: 'G',
  SILVER: 'S',
  BRONZE: 'B'
};


symbols = {
  "❎": -1,
  "0️⃣": 0,
  "1️⃣": 1,
  "2️⃣": 2,
  "3️⃣": 3,
  "4️⃣": 4,
  "5️⃣": 5,
  "6️⃣": 6,
  "7️⃣": 7,
  "8️⃣": 8,
  "9️⃣": 9,
  "🔟": 10,
  "🕚": 11,
  "🕛": 12,
  "🕐": 13,
  "⓮": 14,
  "⓯": 15,
  "🟥": MAX
}


games = {
    WORDLE: {
      "day": date(2021, 6, 19),
      "heading": 'W',
      "json": 'wordle',
      "useBoard": False,
      "ignore": 0,
      "turns": []
    },
    "Wortel": {
      "day": date(2022, 1, 31),
      "heading": '🥕',
      "json": 'wortel',
      "useBoard": False,
      "ignore": 0,
      "turns": []
    },
    "Daily Quordle": {
      "day": date(2022, 1, 24),
      "heading": 'Q',
      "json": 'quordle',
      "useBoard": True,
      "ignore": 0,
      "max": 10,
      "turns": []
    },
    "Daily Sequence Quordle": {
      "day": date(2022, 1, 24),
      "heading": 'SQ',
      "json": 'sequenceQuordle',
      "useBoard": True,
      "ignore": 570,
      "max": 10,
      "turns": []
    },
    "Daily Octordle": {
      "day": date(2022, 1, 24),
      "heading": 'O',
      "json": 'octordle',
      "useBoard": True,
      "ignore": 0,
      "max": 14,
      "turns": []
    },
    "Daily Sequence Octordle": {
      "day": date(2022, 1, 24),
      "heading": 'SO',
      "json": 'sequenceOctordle',
      "useBoard": True,
      "max": 16,
      "turns": [],
      "ignore": 570,
    },
    "nerdlegame": {
      "day": date(2022, 1, 19),
      "heading": 'N',
      "json": 'nerdle',
      "useBoard": False,
      "ignore": 0,
      "turns": []
    }
  }

def calcScore(board, max):
    score = 0
    for s in symbols.keys():
      count = board.count(s)
      score += count * min(max, symbols[s])
    return score

chats = ""
with ZipFile("../data/export.zip", 'r') as zf: 
    ba = zf.read("_chat.txt")
    chats = ba.decode()
    zf.close() 

pattern = r"\[\d+\/\d+\/\d+, \d+:\d+:\d+\] ([A-Za-z é&]+): \s*([ ([A-Za-z 🥕]+) #?(\d,?\d+)( (\d)\/(\d))?([^\[]+)"
matches = re.findall(pattern, chats)
plays = []
for x in matches:
    name = x[0].split()[0]   
    game_name = x[1]
    keys = [key for key, val in games.items() if key in game_name]
    if len(keys) > 0:
        game_key = keys[0]
        game = games[game_key]
        day = int(x[2].replace(",", ""))
        game_date = game["day"] + timedelta(days=day)
        board = ""
        turns = MAX
        if game["useBoard"]:
            board = x[6]
            turns = calcScore(board, game["max"])
        elif x[4]:
            turns = int(x[4])
        plays.append({"date":game_date, "person": name, "heading": game["heading"], "game": game_key, "turns": turns})

# groups plays by date and person
grouped = {}
for p in plays:
    key = p["date"]
    game = p["game"]
    if key in grouped:
        game = grouped[key][game]
        game.append(p)
    else:
        day_games = {key: [] for key in games.keys()}
        day_games[game].append(p)
        grouped[key] = day_games

for day in grouped.keys():
    day_games = grouped[day]
    for game_key in day_games.keys():
        day_plays = day_games[game_key]
        sorted_plays = sorted(day_plays, key=lambda x: x["turns"])
        medal_count = 0;
        medal_index = -1;
        last_turn = 0;
        for p in sorted_plays:
            turns = p["turns"]
            if turns > last_turn:
                if medal_index >= 3:
                    break
                p["medal"] = MEDALS[medal_index]
            last_turn = turns
            medal_index += 1

grouped
